# Comments → CMD 변환기
DB의 comments 테이블을 addDocumentComment 형식의 cmd 파일로 변환

In [1]:
# =============================================================================
# 🔧 설정값 (이관할 때마다 여기만 수정하세요)
# =============================================================================

MIGRATION_NUMBER = 7  # 이관 횟수 (예: 7 → 07로 변환됨)

# CSV 파일 경로
IN_EMPLOYEE_CSV = r"C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관\in_employee.csv"    # 재직자
OUT_EMPLOYEE_CSV = r"C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관\out_employee.csv"  # 퇴사자

# 출력 파일 
OUTPUT_FILE = r"C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관\comments_07.cmds"

# DB 설정
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'any_approval',
    'charset': 'utf8mb4'
}

print(f"이관 횟수: {MIGRATION_NUMBER:02d}")
print(f"재직자 CSV: {IN_EMPLOYEE_CSV}")
print(f"퇴사자 CSV: {OUT_EMPLOYEE_CSV}")
print(f"출력 파일: {OUTPUT_FILE}")

이관 횟수: 07
재직자 CSV: C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관\in_employee.csv
퇴사자 CSV: C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관\out_employee.csv
출력 파일: C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관\comments_07.cmds


In [2]:
# =============================================================================
# 라이브러리 import
# =============================================================================

import pandas as pd
import pymysql
import json
import re

print("라이브러리 로드 완료")

라이브러리 로드 완료


In [3]:
# =============================================================================
# CSV 파일 로드 - 사원명 → ID 매핑 딕셔너리 생성
# =============================================================================

# 재직자 로드 (우선순위 높음)
df_in = pd.read_csv(IN_EMPLOYEE_CSV, encoding='utf-8')
print(f"재직자 로드: {len(df_in)}명")

# 퇴사자 로드
df_out = pd.read_csv(OUT_EMPLOYEE_CSV, encoding='utf-8')
print(f"퇴사자 로드: {len(df_out)}명")

# 매핑 딕셔너리 생성 (퇴사자 먼저 넣고, 재직자로 덮어쓰기 → 재직자 우선)
name_to_id = {}

# 퇴사자 먼저
for _, row in df_out.iterrows():
    if pd.notna(row['사원명']) and pd.notna(row['ID']):
        name_to_id[row['사원명']] = row['ID']

# 재직자로 덮어쓰기 (우선순위)
for _, row in df_in.iterrows():
    if pd.notna(row['사원명']) and pd.notna(row['ID']):
        name_to_id[row['사원명']] = row['ID']

print(f"\n총 매핑 가능 인원: {len(name_to_id)}명")
print("\n[샘플 5개]")
for i, (name, id_) in enumerate(list(name_to_id.items())[:5]):
    print(f"  {name} → {id_}")

재직자 로드: 167명
퇴사자 로드: 336명

총 매핑 가능 인원: 503명

[샘플 5개]
  CEO → temp001
  PHAM NHU QUYNH → temp002
  강대환 → temp003
  강성원 → temp004
  강원준 → temp005


In [4]:
# =============================================================================
# DB 연결 및 comments 테이블 조회
# =============================================================================

conn = pymysql.connect(**DB_CONFIG)

query = "SELECT source_id, source_document_id, created_at, updated_at, writer, message FROM comments"
df_comments = pd.read_sql(query, conn)

conn.close()

print(f"comments 데이터 로드 완료: {len(df_comments)}건")
print(f"\n[샘플 데이터]")
df_comments.head(3)

comments 데이터 로드 완료: 100건

[샘플 데이터]


C:\Users\LEEJUHWAN\AppData\Local\Temp\ipykernel_30204\1543772678.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_comments = pd.read_sql(query, conn)


,source_id,source_document_id,created_at,updated_at,writer,message
0,01_23279081_01,23279081,1720673708000,1720673708000,최지영,"본 건은 MS-SQL 연결작업의 내부처리가 불가하여, 전문업체의 기술지원을 받아 발..."
1,01_26322481_01,26322481,1760585241000,1760585241000,김보라,"시설장비비를 간접비로 6,300만원 전용하여 사용."
2,02_26322481_01,26322481,1760686729000,1760686729000,김보라,시설장비비를 간접비로 630만원 전용하여 사용(6300만원X)


In [5]:
# =============================================================================
# 매핑 검증 - 실패 시 여기서 멈춤!
# =============================================================================

# comments에 있는 모든 writer 추출
writers_in_comments = df_comments['writer'].unique()
print(f"comments 테이블의 고유 writer 수: {len(writers_in_comments)}명")

# 매핑 안 되는 writer 찾기
unmapped_writers = []
for writer in writers_in_comments:
    if writer not in name_to_id:
        count = len(df_comments[df_comments['writer'] == writer])
        unmapped_writers.append((writer, count))

# 결과 출력
if unmapped_writers:
    print("\n" + "="*60)
    print("❌ 매핑 실패! 아래 writer가 CSV에 없습니다.")
    print("="*60)
    for writer, count in sorted(unmapped_writers, key=lambda x: -x[1]):
        print(f"  - {writer} ({count}건)")
    print("\n👉 CSV에 추가하거나 수동 매핑 후 다시 실행하세요.")
    print("="*60)
    raise Exception(f"매핑 실패: {len(unmapped_writers)}명의 writer를 찾을 수 없음")
else:
    print("\n✅ 모든 writer 매핑 성공!")

comments 테이블의 고유 writer 수: 33명

✅ 모든 writer 매핑 성공!


In [6]:
# =============================================================================
# 변환 함수 정의
# =============================================================================

def convert_source_id(original_source_id, migration_num):
    """
    source_id 변환: 01_23447133_01 → 01_23447133_07
    마지막 _XX 부분을 이관 횟수로 교체
    """
    # 마지막 _숫자 부분을 찾아서 교체
    pattern = r'_\d+$'
    new_suffix = f'_{migration_num:02d}'
    return re.sub(pattern, new_suffix, original_source_id)


def convert_source_document_id(original_doc_id, migration_num):
    """
    source_document_id 변환: 23447133 → doc_23447133_07
    """
    return f'doc_{original_doc_id}_{migration_num:02d}'


def escape_message(msg):
    """
    메시지 이스케이프 처리 (JSON 형식용)
    """
    if msg is None:
        return ""
    # 백슬래시 먼저 처리 (순서 중요)
    msg = msg.replace('\\', '\\\\')
    # 따옴표 이스케이프
    msg = msg.replace('"', '\\"')
    # 줄바꿈 이스케이프
    msg = msg.replace('\n', '\\n')
    msg = msg.replace('\r', '\\r')
    # 탭 이스케이프
    msg = msg.replace('\t', '\\t')
    return msg


# 테스트
print("[변환 함수 테스트]")
print(f"source_id: 01_23447133_01 → {convert_source_id('01_23447133_01', MIGRATION_NUMBER)}")
print(f"source_document_id: 23447133 → {convert_source_document_id('23447133', MIGRATION_NUMBER)}")
print(f"message escape: 줄바꿈\\n테스트 → {escape_message('줄바꿈\n테스트')}")

[변환 함수 테스트]
source_id: 01_23447133_01 → 01_23447133_07
source_document_id: 23447133 → doc_23447133_07
message escape: 줄바꿈\n테스트 → 줄바꿈\n테스트


In [7]:
# =============================================================================
# addDocumentComment 형식으로 변환
# =============================================================================

result_lines = []

for idx, row in df_comments.iterrows():
    writer_name = row['writer']
    email_id = name_to_id[writer_name]  # 이미 검증됨
    
    # source_id, source_document_id 변환
    new_source_id = convert_source_id(row['source_id'], MIGRATION_NUMBER)
    new_source_doc_id = convert_source_document_id(row['source_document_id'], MIGRATION_NUMBER)
    
    # 메시지 이스케이프 처리
    escaped_message = escape_message(row['message'])
    
    # JSON 형식 문자열 직접 구성 (줄바꿈 없이 한 줄로)
    cmd = (
        f'addDocumentComment {{'
        f'"sourceId":"{new_source_id}",'
        f'"createdAt":"{row["created_at"]}",'
        f'"writer":{{"emailId":"{email_id}"}},'
        f'"message":"{escaped_message}",'
        f'"sourceDocumentId":"{new_source_doc_id}",'
        f'"updatedAt":"{row["updated_at"]}"'
        f'}}'
    )
    result_lines.append(cmd)

print(f"변환 완료: {len(result_lines)}건")

변환 완료: 100건


In [8]:
# =============================================================================
# 검증 로직
# =============================================================================

print("="*60)
print("검증 결과")
print("="*60)

# 1. 총 건수 확인
print(f"\n[1] 총 건수: {len(result_lines)}건 (DB: {len(df_comments)}건)")
if len(result_lines) == len(df_comments):
    print("    ✅ 일치")
else:
    print("    ❌ 불일치!")

# 2. JSON 유효성 검사
print(f"\n[2] JSON 유효성 검사")
json_errors = []
for i, line in enumerate(result_lines):
    # addDocumentComment 부분 제거하고 JSON만 추출
    json_str = line[len('addDocumentComment '):]
    try:
        json.loads(json_str)
    except json.JSONDecodeError as e:
        json_errors.append((i, str(e)))

if json_errors:
    print(f"    ❌ {len(json_errors)}건 오류 발생!")
    for idx, err in json_errors[:5]:  # 처음 5개만 표시
        print(f"       - 행 {idx}: {err}")
else:
    print(f"    ✅ 모든 JSON 파싱 성공")

# 3. source_id 형식 검증
print(f"\n[3] source_id 형식 검증 (_{MIGRATION_NUMBER:02d} 로 끝나는지)")
suffix_pattern = f'_{MIGRATION_NUMBER:02d}"'
wrong_suffix = [i for i, line in enumerate(result_lines) if suffix_pattern not in line.split('sourceDocumentId')[0]]
if wrong_suffix:
    print(f"    ❌ {len(wrong_suffix)}건 형식 오류")
else:
    print(f"    ✅ 모든 source_id가 _{MIGRATION_NUMBER:02d}로 끝남")

# 4. source_document_id 형식 검증
print(f"\n[4] source_document_id 형식 검증 (doc_로 시작하는지)")
wrong_doc_id = [i for i, line in enumerate(result_lines) if '"sourceDocumentId":"doc_' not in line]
if wrong_doc_id:
    print(f"    ❌ {len(wrong_doc_id)}건 형식 오류")
else:
    print(f"    ✅ 모든 source_document_id가 doc_로 시작함")

print("\n" + "="*60)

검증 결과

[1] 총 건수: 100건 (DB: 100건)
    ✅ 일치

[2] JSON 유효성 검사
    ✅ 모든 JSON 파싱 성공

[3] source_id 형식 검증 (_07 로 끝나는지)
    ✅ 모든 source_id가 _07로 끝남

[4] source_document_id 형식 검증 (doc_로 시작하는지)
    ✅ 모든 source_document_id가 doc_로 시작함



In [9]:
# =============================================================================
# 결과 파일 저장 및 미리보기
# =============================================================================

# 파일 저장
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    f.write('\n'.join(result_lines))

print(f"✅ 완료! {len(result_lines)}건 저장됨: {OUTPUT_FILE}")

# 미리보기
print("\n" + "="*60)
print("미리보기 (처음 3건)")
print("="*60)
for i, line in enumerate(result_lines[:3]):
    print(f"\n[{i+1}]")
    print(line)

print("\n" + "="*60)
print("미리보기 (마지막 2건)")
print("="*60)
for i, line in enumerate(result_lines[-2:]):
    print(f"\n[{len(result_lines)-1+i}]")
    print(line)

✅ 완료! 100건 저장됨: C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관\comments_07.cmds

미리보기 (처음 3건)

[1]
addDocumentComment {"sourceId":"01_23279081_07","createdAt":"1720673708000","writer":{"emailId":"jychoi"},"message":"본 건은 MS-SQL 연결작업의 내부처리가 불가하여, 전문업체의 기술지원을 받아 발생하였습니다.\n프로젝트 완료처리를 위한 필수작업으로 CP에는 반영되지 않은 추가비용입니다.","sourceDocumentId":"doc_23279081_07","updatedAt":"1720673708000"}

[2]
addDocumentComment {"sourceId":"01_26322481_07","createdAt":"1760585241000","writer":{"emailId":"kimbr"},"message":"시설장비비를 간접비로 6,300만원 전용하여 사용.","sourceDocumentId":"doc_26322481_07","updatedAt":"1760585241000"}

[3]
addDocumentComment {"sourceId":"02_26322481_07","createdAt":"1760686729000","writer":{"emailId":"kimbr"},"message":"시설장비비를 간접비로 630만원 전용하여 사용(6300만원X)","sourceDocumentId":"doc_26322481_07","updatedAt":"1760686729000"}

미리보기 (마지막 2건)

[99]
addDocumentComment {"sourceId":"01_14884099_07","createdAt":"1605595960000","writer":{"emailId":"stmpst"},"message":"최종 결재자는 승인이 되어야 합니다","sourceDocumentId"